# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from datetime import datetime



In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()



In [3]:
# getting file path 
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
# defining folder path and reading all the file path in dict
song_files = get_files("data/song_data")

In [5]:
song_files[0:2]

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json']

In [6]:
# reading record at zero position
filepath = song_files[0]

In [7]:
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [8]:
# reading file at zero position in dict
df = pd.read_json(filepath,  lines=True)

In [9]:
df


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [10]:
# dropping all unnecessary columns
df = df.drop(['artist_latitude', 'artist_location', 'artist_longitude', 'artist_name',  'num_songs'], axis=1)
df.head()

,artist_id,duration,song_id,title,year
0,ARD7TVE1187B99BFB1,218.93179,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
#converting data to list format
song_data=df.values[0].tolist()
#song_data = df.values
song_data

['ARD7TVE1187B99BFB1', 218.93179, 'SOMZWCG12A8C13C480', "I Didn't Mean To", 0]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
# executing query and adding data , single record to the table
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
df

,artist_id,duration,song_id,title,year
0,ARD7TVE1187B99BFB1,218.93179,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [14]:
# reading record number 7
# dropping unwanted columns

artist_data = pd.read_json(song_files[7],  lines=True)
print(artist_data)
artist_data = artist_data.drop(["duration", "num_songs", "song_id", "title", "year"], axis=1)

print(artist_data)



            artist_id  artist_latitude artist_location  artist_longitude  \
0  ARGSJW91187B9B1D6B         35.21962  North Carolina         -80.01955   

    artist_name   duration  num_songs             song_id            title  \
0  JennyAnyKind  218.77506          1  SOQHXMF12AB0182363  Young Boy Blues   

   year  
0     0  
            artist_id  artist_latitude artist_location  artist_longitude  \
0  ARGSJW91187B9B1D6B         35.21962  North Carolina         -80.01955   

    artist_name  
0  JennyAnyKind  


In [15]:
artist_data

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name
0,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind


In [16]:
#converting data to list format
artist_data=artist_data.values[0].tolist()
artist_data

['ARGSJW91187B9B1D6B', 35.21962, 'North Carolina', -80.01955, 'JennyAnyKind']

In [17]:
artist_data

['ARGSJW91187B9B1D6B', 35.21962, 'North Carolina', -80.01955, 'JennyAnyKind']

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [18]:
# executing query and adding data, single record to the table
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [19]:
# defining log file destination and reading file path 
log_files = get_files("data/log_data")
logfilepath = log_files[0]
song_data = pd.read_json(logfilepath,  lines=True)
# displaying record from 0 to 1
song_data[0:2]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
# filtering out the column with desired string
df= song_data.loc[song_data['page'] ==  "NextSong"]

In [21]:
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [22]:
# converting ts column from UNIX format timestamp to human readable format 
# also putting year, month, week, day, week number, min, sec hour in sapeate columns
list_timestamp = []
list_year = []
list_month = []
list_date = []
list_hour = []
list_minute = []
list_second = []
list_weekofyear = []
list_weekday = []

for td in df.ts:
    
    timestamp = (datetime.utcfromtimestamp(td/1000).strftime('%Y-%m-%d %H:%M:%S'))
    year = (datetime.utcfromtimestamp(td/1000).strftime('%Y'))
    month = (datetime.utcfromtimestamp(td/1000).strftime('%m'))
    date = (datetime.utcfromtimestamp(td/1000).strftime('%d'))
    hour = (datetime.utcfromtimestamp(td/1000).strftime('%H'))
    minute = (datetime.utcfromtimestamp(td/1000).strftime('%M'))
    second = (datetime.utcfromtimestamp(td/1000).strftime('%S'))
    weekofyear = (datetime.utcfromtimestamp(td/1000).strftime('%U'))
    weekday = (datetime.utcfromtimestamp(td/1000).strftime('%A'))
    

    list_timestamp.append(timestamp)
    list_year.append(year)
    list_month.append(month)
    list_date.append(date)
    list_hour.append(hour)
    list_minute.append(minute)
    list_second.append(second)
    list_weekofyear.append(weekofyear)
    list_weekday.append(weekday)
df["normal_ts"] = list_timestamp
df["year"] = list_year
df["month"] = list_month
df["date"] = list_date
df["hour"] = list_hour
df["min"] = list_minute
df["sec"] = list_second
df["weekofyear"] = list_weekofyear
df["weekdayname"] = list_weekday


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [23]:
# dropping all unnecessary columns
dftemp = df.drop(["artist","auth","firstName","gender","itemInSession","lastName","length","level","location","method","page","registration","sessionId","song","status","userAgent","userId"], axis=1)


In [24]:
dftemp.head()

,ts,normal_ts,year,month,date,hour,min,sec,weekofyear,weekdayname
0,1543537327796,2018-11-30 00:22:07,2018,11,30,00,22,07,47,Friday
1,1543540121796,2018-11-30 01:08:41,2018,11,30,01,08,41,47,Friday
2,1543540368796,2018-11-30 01:12:48,2018,11,30,01,12,48,47,Friday
3,1543540625796,2018-11-30 01:17:05,2018,11,30,01,17,05,47,Friday
4,1543540856796,2018-11-30 01:20:56,2018,11,30,01,20,56,47,Friday


In [25]:
# dropping all unnecessary columns
time_df = dftemp.drop(["normal_ts","min","sec"], axis=1)


In [26]:
time_df.head()

,ts,year,month,date,hour,weekofyear,weekdayname
0,1543537327796,2018,11,30,00,47,Friday
1,1543540121796,2018,11,30,01,47,Friday
2,1543540368796,2018,11,30,01,47,Friday
3,1543540625796,2018,11,30,01,47,Friday
4,1543540856796,2018,11,30,01,47,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [27]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [28]:
user_df=  song_data.loc[song_data['page'] ==  "NextSong"]

In [29]:
# dropping all unnecessary columns
user_df = user_df.drop(["ts","artist","auth","itemInSession","length","location","method","page","registration","sessionId","song","status","userAgent"], axis=1)


In [30]:
user_df.head()

,firstName,gender,lastName,level,userId
0,Jayden,M,Bell,free,91
1,Jacob,M,Klein,paid,73
2,Jacob,M,Klein,paid,73
3,Jacob,M,Klein,paid,73
4,Jacob,M,Klein,paid,73


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
user_df.head()

,firstName,gender,lastName,level,userId
0,Jayden,M,Bell,free,91
1,Jacob,M,Klein,paid,73
2,Jacob,M,Klein,paid,73
3,Jacob,M,Klein,paid,73
4,Jacob,M,Klein,paid,73


In [32]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [33]:
# filtering out column with desired string
dflog= song_data.loc[song_data['page'] ==  "NextSong"]

In [34]:
dflog.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [35]:
for index, row in dflog.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    #print(results)
    
    if results:
        songid, artistid = results
        #print(songid)
        #print(artistid)
    else:
        songid, artistid = None, None

    songplay_data = (row.ts, 
                     row.userId, 
                     row.level, 
                     songid, 
                     artistid, 
                     row.sessionId, 
                     row.location, 
                     row.userAgent)
    #len(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [36]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.